TensorFlow opublikował Speech Commands Datasets. Baza danych zawiera 65 tysięcy jednosekundowych nagrań 30 krótkich słów w języku angielskim, wypowiedzianych przez kilka ttsięcy różnych osób. W trakcie laboratorium zostaną użyte głębokie sieci neuronowe do rozpoznawania tych słów.

Baza nagrań może być pobrana stąd [tutaj](https://www.kaggle.com/c/tensorflow-speech-recognition-challenge).

**Implementacja modelu Speech-to-Text w Python'ie od podstaw**

**Import niezbędnych bibliotek**

W pierwszej kolejności importujemy biblioteki niezbędne do przetwarzania analizowanych danych. LibROSA i SciPy są bibiotekami Python'a używanymi do przetwarzania sygnałów audio.

In [ ]:
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import warnings

warnings.filterwarnings("ignore")

In [ ]:
os.listdir('/home/ilona/Desktop/Literatura/SemestrVII/Systemy_Glosowe/voice_sys/AI_voice_recognition
')

**Eksploracja i wizualizacja danych**

Eksploracja i wizualizacja danych pomaga zrozumieć zawartość danych oraz transformację danych w trakcie procesu ich przetwarzania. 

**Wizualizacja sygnału audio w dziedzinie czasu**


In [ ]:
train_audio_path = '../input/tensorflow-speech-recognition-challenge/train/audio/'
samples, sample_rate = librosa.load(train_audio_path+'yes/0a7c2a8d_nohash_0.wav', sr = 16000)
fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
ax1.set_title('Raw wave of ' + '../input/train/audio/yes/0a7c2a8d_nohash_0.wav')
ax1.set_xlabel('time')
ax1.set_ylabel('Amplitude')
ax1.plot(np.linspace(0, sample_rate/len(samples), sample_rate), samples)

**Częstotliwość próbkowania **

Sprawdźmy, jaka jest częstotliwość próbkowania sygnału audio

In [ ]:
ipd.Audio(samples, rate=sample_rate)

In [ ]:
print(sample_rate)

**Resampling**

Z informacji zwróconej przez funkcję load z biblioteki LibROSA wynika, że częstotliwośc próbkowania nagrań to 16000 Hz. Wykonamy teraz resamplig sygnału usuwając z sygnału częstotliwości powyżej 8000 Hz. Głowym celem tego zabiegu jest ograniczenie czasu treningu sieci neuronowej. Częstotliwości słyszalne przez ludzkie ucho mieszczą się w przedziale do ok. 22 000 Hz.

In [ ]:
samples = librosa.resample(samples, sample_rate, 8000)
ipd.Audio(samples, rate=8000)

 A teraz sprawdźmy ile jest w bazie danych nagrań dla każdego polecenia głosowego:

In [ ]:
labels=os.listdir(train_audio_path)

In [ ]:
#dla każdej etykiety z listy kartotek znajdujemy liczbę plików audio w tej kartotece
no_of_recordings=[]
for label in labels:
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    no_of_recordings.append(len(waves))
    
#wykres
plt.figure(figsize=(30,5))
index = np.arange(len(labels))
plt.bar(index, no_of_recordings)
plt.xlabel('Commands', fontsize=12)
plt.ylabel('No of recordings', fontsize=12)
plt.xticks(index, labels, fontsize=15, rotation=60)
plt.title('No. of recordings for each command')
plt.show()

In [ ]:
labels=["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]

**Długość nagrań**

A teraz sprawdźmy, jak długie są nagrania poleceń głosowych:

In [ ]:
duration_of_recordings=[]
for label in labels:
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        sample_rate, samples = wavfile.read(train_audio_path + '/' + label + '/' + wav)
        duration_of_recordings.append(float(len(samples)/sample_rate))
    
plt.hist(np.array(duration_of_recordings))

**Preprocessing sygnałów audio**

W efekcie przeprowadzonej eksploracji danych ustaliliśmy, że w przypadku niektórych nagrań ich długość jest mniejsza, niż jedna sekunda oraz, że częstotliwość próbkowania wynosi 16 tys Hz. Spośród wszystkich nagrań do dalszej pracy wybieramy tylko te z komendami zapisanymi w tablicy labels. Wykonujemy resampling nagrań do częstotliwości 8 tys HZ i pozostawiamy tylko nagrania jednosekundowe (złożone z 8000 próbek).


Kod realizujący założone kroki preprocessingu:

In [ ]:
train_audio_path = '../input/tensorflow-speech-recognition-challenge/train/audio/'

all_wave = []
all_label = []
for label in labels:
    print(label)
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples)== 8000) : 
            all_wave.append(samples)
            all_label.append(label)

Konwertujemy etykiety tekstowe do kodów całkowitoliczbowych, wymaganych do treningu klasyfikatora:

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)

Zaczyna się magia frameworku keras...
Konwertujemy całkowitoliczbowe etykiery do jednego wektora:

In [ ]:
from keras.utils import np_utils
y=np_utils.to_categorical(y, num_classes=len(labels))

Przerabiamy macierze 2D na 3D ponieważ takiego wejścia spodziewa się sieć, która będzie użyta do rozpoznawania komend głosowych:

In [ ]:
all_wave = np.array(all_wave).reshape(-1,8000,1)

**Wybór zbioru treningowego i walidacyjnego**

Użyjemy 80% danych do treningu, a pozostałe 20% do walidacji wytrenowanego modelu:


In [ ]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(np.array(all_wave),np.array(y),stratify=y,test_size = 0.2,random_state=777,shuffle=True)

**Wybór architektury sieci dla problemu rozpozawaia komed głosowych**

Model speech-to-text zostanie zbudowany w oparciu o sieć conv1d. Conv1d to konwolucyjna sieć neuronowa, wykonująca konwolucję tylko w jedym wymiarze. 

**Budowa modelu**

Budujemy model korzystając z API frameworku Keras.

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
K.clear_session()

inputs = Input(shape=(8000,1))

#First Conv1D layer
conv = Conv1D(8,13, padding='valid', activation='relu', strides=1)(inputs)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Second Conv1D layer
conv = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Third Conv1D layer
conv = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Fourth Conv1D layer
conv = Conv1D(64, 7, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Flatten layer
conv = Flatten()(conv)

#Dense Layer 1
conv = Dense(256, activation='relu')(conv)
conv = Dropout(0.3)(conv)

#Dense Layer 2
conv = Dense(128, activation='relu')(conv)
conv = Dropout(0.3)(conv)

outputs = Dense(len(labels), activation='softmax')(conv)

model = Model(inputs, outputs)
model.summary()

Definiujemy "loss function" w oparciu o "categorical cross-entropy" - wybór odpowiedni dla problemu klasyfikacji wieloklasowej:

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

"Early stopping" i "model checkpoints" to callback'i użyte w celu zatrzymania treningu sieci neuronowej w odpowiednim momencie (zanim zacznie się przetrenowanie) i w celu zapisania najlepszego uzyskanego modelu uzyskanego po każdej z epok treingu:

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('best_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

Trening i testowaie na zbiorze walidacyjnym:

In [ ]:
history=model.fit(x_tr, y_tr ,epochs=3, callbacks=[es,mc], batch_size=32, validation_data=(x_val,y_val))

**Wykres diagnostyczny**

Wizualizacja pomaga nam zrozumieć, jak działa sieć na danych treningiwych, a jak na walidacyjnych:

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

**Ładujemy najlepszy zapamiętany model**

In [ ]:
from keras.models import load_model
model=load_model('best_model.hdf5')

Definiujemy funkcję, która znajduje wyraz odpowiadający sygnałowi audio::

In [ ]:
def predict(audio):
    prob=model.predict(audio.reshape(-1,8000,1))
    index=np.argmax(prob[0])
    return classes[index]

**Predykcja**
Sprawdzamy działanie wytrenowanej sieci na zbiorze walidacyjnym (poniższy skrypt można uruchamiać wielokrotnie):

In [ ]:
import random
index=random.randint(0,len(x_val)-1)
samples=x_val[index].ravel()
print("Audio:",classes[np.argmax(y_val[index])])
ipd.Audio(samples, rate=8000)

In [ ]:
print("Text:",predict(samples))

Poniższy skrypt prosi użytkownika o nagranie komendy głosowej. Proszę nagrać jedną z komend rozpoznawanych przez wytrenowaną sieć:

In [ ]:
import sounddevice as sd
import soundfile as sf

samplerate = 16000  
duration = 1 # seconds
filename = 'stop.wav'
print("start")
mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,
    channels=1, blocking=True)
print("end")
sd.wait()
sf.write(filename, mydata, samplerate)

Wczytajmy zapisany plik audio i sprawdźmy, jak radzi sobie z nim wytrenowana sieć:

In [ ]:
os.listdir('../input')

In [ ]:
filepath='../input'

In [ ]:
#reading the voice commands
samples, sample_rate = librosa.load(filepath + '/' + 'stop.wav', sr = 16000)
samples = librosa.resample(samples, sample_rate, 8000)
ipd.Audio(samples,rate=8000)              

In [ ]:
#converting voice commands to text
predict(samples)

Gratulacje! A teraz proszę sprawdzić skuteczność działania sieci na całym zbiorze walidacyjnym...